# Minimal OpenSCM-Runner example with FaIR

Requires fair >= 1.6.0a (to be released) and < 2.0.0

In [1]:
import openscm_runner

In [2]:
# NBVAL_IGNORE_OUTPUT
print(openscm_runner.__version__)

0.1.0-alpha.1+20.g3fe63c6.dirty


In [3]:
import pandas as pd
import pyam
import numpy as np
import matplotlib.pyplot as pl

from tqdm import tqdm_notebook

from openscm_runner.run import run
from openscm_runner.adapters import FAIR
from fair.tools.scmdf import scmdf_to_emissions

from fair.forward import fair_scm

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler
/nfs/b0110/Users/mencsm/openscm-runner/src/openscm_runner/run.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
fair=FAIR()

In [5]:
fair.get_version()

'1.5.1+5.g80440d2.dirty'

In [6]:
df = pd.read_csv('../data/tiny_example_emissions_harmonized_infilled.csv')
df = pyam.IamDataFrame(df)
df_clean = df.filter(variable="*Infilled*")
df_fair = pyam.IamDataFrame(df_clean.timeseries().reset_index())

In [7]:
x = run(
    climate_models_cfgs={
            "FAIR": [{}],   # passing an empty list of an empty dict will run with defaults
        },
    scenarios=df_fair,
    output_variables=('Surface Temperature',)
)

In [8]:
x.timeseries()

time                                                  1765-01-01  1766-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       0.006648     0.01256   
model2 1point5  unspecified Surface Temperature K       0.006648     0.01256   
       2point0  unspecified Surface Temperature K       0.006648     0.01256   

time                                                  1767-01-01  1768-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       0.018838    0.028964   
model2 1point5  unspecified Surface Temperature K       0.018838    0.028964   
       2point0  unspecified Surface Temperature K       0.018838    0.028964   

time                                                  1769-01-01  1770-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       0.040336    0.052402   
model2 1point5  unspecified Surface Temperature K       0.040336    0.052402   
       2point0  unspecified Surface Temperature K       0.040336    0.052402   

time                                                  1771-01-01  1772-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       0.059805    0.061209   
model2 1point5  unspecified Surface Temperature K       0.059805    0.061209   
       2point0  unspecified Surface Temperature K       0.059805    0.061209   

time                                                  1773-01-01  1774-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       0.062393     0.06282   
model2 1point5  unspecified Surface Temperature K       0.062393     0.06282   
       2point0  unspecified Surface Temperature K       0.062393     0.06282   

time                                                  ...  2091-01-01  \
model  scenario region      variable            unit  ...               
model1 1point5  unspecified Surface Temperature K     ...    1.557889   
model2 1point5  unspecified Surface Temperature K     ...    1.573470   
       2point0  unspecified Surface Temperature K     ...    1.655035   

time                                                  2092-01-01  2093-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       1.566841    1.572818   
model2 1point5  unspecified Surface Temperature K       1.581457    1.586486   
       2point0  unspecified Surface Temperature K       1.663196    1.668396   

time                                                  2094-01-01  2095-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       1.575093    1.575041   
model2 1point5  unspecified Surface Temperature K       1.587807    1.586774   
       2point0  unspecified Surface Temperature K       1.669886    1.669020   

time                                                  2096-01-01  2097-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       1.572774    1.570512   
model2 1point5  unspecified Surface Temperature K       1.583489    1.580161   
       2point0  unspecified Surface Temperature K       1.665900    1.662736   

time                                                  2098-01-01  2099-01-01  \
model  scenario region      variable            unit                           
model1 1point5  unspecified Surface Temperature K       1.568306    1.568151   
model2 1point5  unspecified Surface Temperature K       1.576840    1.575516   
       2point0  unspecified Surface Temperature K       1.659575    1.658411   

time                     